In [ ]:
#  Data Wrangling on Real Estate Market 

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# 1. Import the dataset and clean column names
df = pd.read_csv("Bengaluru_House_Data.csv")
df.columns = df.columns.str.replace(r"[^a-zA-Z0-9]", "_").str.lower()  # Replace special characters, spaces, make lowercase
print("Cleaned Columns:", df.columns)

Cleaned Columns: Index(['area_type', 'availability', 'location', 'size', 'society',
       'total_sqft', 'bath', 'balcony', 'price'],
      dtype='object')


In [8]:
df.head()

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,GrrvaGr,1440,2.0,3.0,62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,95.00
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,GrrvaGr,1200,2.0,1.0,51.00


In [3]:
# 2. Handle missing values
# Check missing values
print(df.isnull().sum())

area_type          0
availability       0
location           1
size              16
society         5502
total_sqft         0
bath              73
balcony          609
price              0
dtype: int64


In [4]:
# Decide on imputation or removal strategy
# Example: Impute missing values in numerical columns with the median
numerical_cols = df.select_dtypes(include=['float64', 'int64']).columns
df[numerical_cols] = df[numerical_cols].apply(lambda x: x.fillna(x.median()))


In [5]:

# For categorical columns, fill with the most frequent value
categorical_cols = df.select_dtypes(include=['object']).columns
df[categorical_cols] = df[categorical_cols].apply(lambda x: x.fillna(x.mode()[0]))

In [6]:
# 3. Perform data merging (assuming availability of an additional dataset for demo purposes)
# Load additional dataset, e.g., Neighborhood info (if available)
# neighborhood_df = pd.read_csv("Neighborhood_Info.csv")
# df = pd.merge(df, neighborhood_df, on="neighborhood_id", how="left")


In [10]:
# Example 1: Filter data where `area_type` is 'Super built-up Area'
super_built_up = df[df['area_type'] == 'Super built-up Area']


In [11]:
# Display filtered data
print(super_built_up)

Empty DataFrame
Columns: [area_type, availability, location, size, society, total_sqft, bath, balcony, price]
Index: []


In [13]:
# Example 2: Filter properties with a minimum of 3 bedrooms
three_bedroom_properties = df[df['size'].str.contains('3 BHK|3 Bedroom')]
print(three_bedroom_properties)

                  area_type   availability            location       size  \
2            Built-up  Area  Ready To Move         Uttarahalli      3 BHK   
3      Super built-up  Area  Ready To Move  Lingadheeranahalli      3 BHK   
8      Super built-up  Area  Ready To Move        Marathahalli      3 BHK   
10     Super built-up  Area         18-Feb          Whitefield      3 BHK   
14               Plot  Area  Ready To Move            Sarjapur  3 Bedroom   
...                     ...            ...                 ...        ...   
13307        Built-up  Area  Ready To Move        Billekahalli      3 BHK   
13308        Built-up  Area  Ready To Move   Bannerghatta Road      3 BHK   
13309  Super built-up  Area  Ready To Move        Yeshwanthpur      3 BHK   
13313  Super built-up  Area  Ready To Move         Uttarahalli      3 BHK   
13314  Super built-up  Area  Ready To Move   Green Glen Layout      3 BHK   

       society total_sqft  bath  balcony   price  
2      GrrvaGr       144

In [15]:
# Example 4: Properties with a price below 50 Lakhs
budget_properties = df[df['price'] < 50]
print(budget_properties)

                  area_type   availability                  location  \
0      Super built-up  Area         19-Dec  Electronic City Phase II   
5      Super built-up  Area  Ready To Move                Whitefield   
12     Super built-up  Area  Ready To Move        7th Phase JP Nagar   
13           Built-up  Area  Ready To Move                 Gottigere   
16     Super built-up  Area  Ready To Move             Bisuvanahalli   
...                     ...            ...                       ...   
13291            Plot  Area         18-Jan            Weavers Colony   
13302  Super built-up  Area  Ready To Move      Annaiah Reddy Layout   
13304  Super built-up  Area  Ready To Move     Raja Rajeshwari Nagar   
13312  Super built-up  Area  Ready To Move                 Bellandur   
13319  Super built-up  Area  Ready To Move              Doddathoguru   

            size  society total_sqft  bath  balcony  price  
0          2 BHK  Coomee        1056   2.0      1.0  39.07  
5          2 

In [16]:
# 5. Handle categorical variables by encoding them
# Label encode binary categorical columns
binary_cols = [col for col in categorical_cols if df[col].nunique() == 2]
for col in binary_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])


In [17]:
# One-hot encode multi-class categorical variables
df = pd.get_dummies(df, columns=[col for col in categorical_cols if col not in binary_cols])


In [23]:
# 7. Identify and handle outliers
# Use IQR to detect outliers in the 'sale_price' column
Q1 = df['price'].quantile(0.25)
Q3 = df['price'].quantile(0.75)
IQR = Q3 - Q1
outliers = df[(df['price'] < (Q1 - 1.5 * IQR)) | (df['price'] > (Q3 + 1.5 * IQR))]
print(f"Number of outliers detected: {outliers.shape[0]}")

Number of outliers detected: 1276


In [25]:
# Optionally, remove outliers
df = df[~((df['price'] < (Q1 - 1.5 * IQR)) | (df['price'] > (Q3 + 1.5 * IQR)))]
